# Part III:  

#### Fix fields
- lost keyword and one other field; restore by inner merge on deduped conversation_id
- evaluate fields with no analytical value and clean up (drop those w/o value).
- 



In [9]:
import os
import pandas as pd
from datetime import datetime

In [30]:
%%sh
ls -lrt ../../data | grep norm

-rw-r--r--  1 melissacirtain  staff  17710489 Apr  2 21:51 combined_model_data_without_annotations_norm_2023-04-02_20.13.51.914670.tsv
-rw-r--r--  1 melissacirtain  staff  17888142 Apr  2 21:51 combined_model_data_all_norm_2023-04-02_20.13.51.914670.tsv
-rw-r--r--  1 melissacirtain  staff    178306 Apr  2 21:51 combined_model_data_with_annotations_norm_2023-04-02_20.13.51.914670.tsv
-rw-r--r--  1 melissacirtain  staff  18067647 Apr  7 07:55 normalized_annotated_all_2023-04-07_07.55.28.945037_OBSOLETE.tsv
-rw-r--r--  1 melissacirtain  staff  18083911 Apr  7 13:01 normalized_annotated_all_2023-04-07_12.43.49.255973.tsv
-rw-r--r--  1 melissacirtain  staff  18083911 Apr  8 13:21 normalized_annotatted_all_2023-04-07_13.21.00.0000.tsv


In [23]:
df = pd.read_csv('../../data/normalized_annotatted_all_2023-04-07_13.21.00.0000.tsv', sep='\t')
print(df.shape, df.columns)

# We lost keyword.  Are we missing anything else?
keys_df = pd.read_json('../../data/project_data.2023-03-11_13.11.55.783862.json')
print('\nMissing Fields:', [x for x in keys_df.columns if x not in df.columns])

(30611, 37) Index(['Unnamed: 0', 'Unnamed: 0.1', 'conversation_id', 'lang',
       'reply_settings', 'created_at', 'clean_text', 'text', 'author_id',
       'referenced_tweets', 'id', 'edit_history_tweet_ids',
       'public_metrics.retweet_count', 'public_metrics.reply_count',
       'public_metrics.like_count', 'public_metrics.impression_count',
       'in_reply_to_user_id', 'geo.place_id', 'withheld.copyright',
       'withheld.country_codes', 'geo.coordinates.type',
       'geo.coordinates.coordinates', 'textblob_Polarity',
       'textblob_Subjectivity', 'vader_Polarity', 'vader_Subjectivity',
       'AFINN_scores', 'AFINN_sentiments', 'SentiWordNet_scores',
       'normalized_textblob_Polarity', 'normalized_textblob_Subjectivity',
       'normalized_vader_Polarity', 'normalized_vader_Subjectivity',
       'normalized_AFINN_scores', 'normalized_SentiWordNet_scores',
       'our_label', 'annotator'],
      dtype='object')

Missing Fields: ['public_metrics.quote_count', 'keyword']


### Recover lost fields

In [28]:
print(keys_df.shape)
keys_df.drop_duplicates(['conversation_id'], keep='first', inplace=True)
print(keys_df.shape)

keys_df.conversation_id.nunique()

(36581, 21)
(30611, 21)


30611

In [29]:
# Get keywords back
print(df.shape)
df = df.merge(right=keys_df[['conversation_id', 'keyword', 'public_metrics.quote_count']], 
              how='left', 
              on='conversation_id')

print(df.shape)
df.head()

(30611, 37)
(30611, 39)


,Unnamed: 0,Unnamed: 0.1,conversation_id,lang,reply_settings,created_at,clean_text,text,author_id,referenced_tweets,...,normalized_textblob_Polarity,normalized_textblob_Subjectivity,normalized_vader_Polarity,normalized_vader_Subjectivity,normalized_AFINN_scores,normalized_SentiWordNet_scores,our_label,annotator,keyword,public_metrics.quote_count
0,0,0,1633954063934009344,en,everyone,2023-03-09 22:13:00+00:00,rt infantry bucky lucky chrisrocklive worse th...,RT @Infantry_bucky: He’s lucky a #ChrisRockLiv...,1519164980582653952,"[{'type': 'retweeted', 'id': '1633938373529292...",...,0.480556,0.527778,0.176602,0.640625,0.448980,0.444444,NaN,NaN,#chrisrocklive,0
1,1,1,1633954058212876288,en,everyone,2023-03-09 22:12:59+00:00,rt ofakindnocap chris rock cheated never perso...,RT @1_ofakindnocap: Chris Rock: “we all been c...,21575184,"[{'type': 'retweeted', 'id': '1632283297588948...",...,0.500000,0.000000,0.217017,0.529018,0.367347,0.333333,NaN,NaN,#chrisrocklive,0
2,2,2,1633951267423768576,en,everyone,2023-03-09 22:01:54+00:00,rt ofakindnocap chris rock cheated never perso...,RT @1_ofakindnocap: Chris Rock: “we all been c...,360633018,"[{'type': 'retweeted', 'id': '1632283297588948...",...,0.500000,0.000000,0.217017,0.529018,0.367347,0.333333,NaN,NaN,#chrisrocklive,0
3,3,3,1633950853626318848,en,everyone,2023-03-09 22:00:15+00:00,rt rolandsmartin working watching chrisrock ne...,RT @rolandsmartin: Working out and watching th...,1547101103803830272,"[{'type': 'retweeted', 'id': '1632825595473149...",...,0.623377,0.535714,0.702710,0.191964,0.489796,0.412698,NaN,NaN,#chrisrocklive,0
4,4,4,1633950824664645632,en,everyone,2023-03-09 22:00:08+00:00,rt rolandsmartin workout done comment chrisroc...,RT @rolandsmartin: Workout done. I’ll have a f...,1547101103803830272,"[{'type': 'retweeted', 'id': '1632833853021728...",...,0.623377,0.535714,0.702710,0.219866,0.489796,0.412698,NaN,NaN,#chrisrocklive,0


### Which columns can we drop?


In [38]:
for c in df.columns:
    # "".join(word.ljust(col_width) for word in row)
    print(c.ljust(35), str(df[c].nunique()).ljust(10))
    #print(f"{c: >25} {df[c].nunique: >20}")

Unnamed: 0                          30611     
Unnamed: 0.1                        30611     
conversation_id                     30611     
lang                                1         
reply_settings                      3         
created_at                          19444     
clean_text                          12763     
text                                12793     
author_id                           25604     
referenced_tweets                   9131      
id                                  30610     
edit_history_tweet_ids              30607     
public_metrics.retweet_count        807       
public_metrics.reply_count          17        
public_metrics.like_count           79        
public_metrics.impression_count     692       
in_reply_to_user_id                 3025      
geo.place_id                        101       
withheld.copyright                  1         
withheld.country_codes              6         
geo.coordinates.type                1         
geo.coordinat

In [49]:
df['withheld.country_codes'].value_counts() # number of different country codes, 46 rows total
df['geo.place_id'].nunique()  # 101 unique, 126 rows total

df.drop(['Unnamed: 0', 'Unnamed: 0.1',
          'lang', 'id', 'withheld.copyright',
          'withheld.country_codes', 'geo.place_id'],
        inplace=True, axis=1)

print(df.shape, df.columns)

(30611, 32) Index(['conversation_id', 'reply_settings', 'created_at', 'clean_text', 'text',
       'author_id', 'referenced_tweets', 'edit_history_tweet_ids',
       'public_metrics.retweet_count', 'public_metrics.reply_count',
       'public_metrics.like_count', 'public_metrics.impression_count',
       'in_reply_to_user_id', 'geo.coordinates.type',
       'geo.coordinates.coordinates', 'textblob_Polarity',
       'textblob_Subjectivity', 'vader_Polarity', 'vader_Subjectivity',
       'AFINN_scores', 'AFINN_sentiments', 'SentiWordNet_scores',
       'normalized_textblob_Polarity', 'normalized_textblob_Subjectivity',
       'normalized_vader_Polarity', 'normalized_vader_Subjectivity',
       'normalized_AFINN_scores', 'normalized_SentiWordNet_scores',
       'our_label', 'annotator', 'keyword', 'public_metrics.quote_count'],
      dtype='object')


# Inter-annotator Agreement

### Reference:
https://psych.unl.edu/psycrs/handcomp/hckappa.PDF 

In [59]:
score_cols = ['normalized_textblob_Polarity','normalized_vader_Polarity',
       'normalized_AFINN_scores', 'normalized_SentiWordNet_scores']
#df.loc[df[score_cols] >= 0]
df[score_cols].describe()  # make sure they're all centered at zero, yes


,normalized_textblob_Polarity,normalized_vader_Polarity,normalized_AFINN_scores,normalized_SentiWordNet_scores
count,30611.000000,30611.000000,30611.000000,30611.000000
mean,0.530875,0.507008,0.485524,0.422334
std,0.121679,0.225197,0.060486,0.061745
min,0.000000,0.000000,0.000000,0.000000
25%,0.500000,0.360800,0.448980,0.396825
50%,0.500000,0.499164,0.489796,0.412698
75%,0.568182,0.649076,0.510204,0.444444
max,1.000000,1.000000,1.000000,1.000000


For strictly > .5, (1583, 32); < .5, (3303, 32)

For >= .5, (1753, 32); for <= .5, (12200, 32)

In [63]:
df.loc[
    (df['normalized_textblob_Polarity'] < 0.5) &
    (df['normalized_vader_Polarity'] < 0.5) &
    (df['normalized_AFINN_scores'] < 0.5) &
    (df['normalized_SentiWordNet_scores'] < 0.5)
].shape

(3303, 32)

### Found a correlation

Looks like AFINN and vader are highly correlated.  Next, look at their inter-annotator agreement over 0.5

In [64]:
df[score_cols].corr()

,normalized_textblob_Polarity,normalized_vader_Polarity,normalized_AFINN_scores,normalized_SentiWordNet_scores
normalized_textblob_Polarity,1.000000,0.440644,0.461542,0.377246
normalized_vader_Polarity,0.440644,1.000000,0.827546,0.390117
normalized_AFINN_scores,0.461542,0.827546,1.000000,0.388647
normalized_SentiWordNet_scores,0.377246,0.390117,0.388647,1.000000


In [84]:
# Make the above more printable/publishable
temp_df = df[score_cols]
temp_df.columns = ['textblob','vader','afinn','sentiwordnet']
temp_df.corr()

,textblob,vader,afinn,sentiwordnet
textblob,1.000000,0.440644,0.461542,0.377246
vader,0.440644,1.000000,0.827546,0.390117
afinn,0.461542,0.827546,1.000000,0.388647
sentiwordnet,0.377246,0.390117,0.388647,1.000000


### Cohen's Kappa at 0.5 boundary

- Making this a binary classification about 0.5
- Probability of same classification by chance = 0.5

In [65]:
from sklearn.metrics import cohen_kappa_score
cohen_kappa_score()

In [67]:
y1 = (df['normalized_AFINN_scores'] > 0.5)
y2 = (df['normalized_vader_Polarity'] > 0.5)

cohen_kappa_score(y1, y2)

0.5553993980731224

In [71]:
# Run cohen's kappa binary classifier eval over all model pairs

from itertools import combinations
for a, b in combinations(score_cols, 2):
    y1 = (df[a] <= 0.5)
    y2 = (df[b] <= 0.5)
    score = cohen_kappa_score(y1, y2)
    print(f'Model A: {a}   Model B: {b}   {score}\n')
    

Model A: normalized_textblob_Polarity   Model B: normalized_vader_Polarity   0.2039724225700278

Model A: normalized_textblob_Polarity   Model B: normalized_AFINN_scores   0.3937684070430598

Model A: normalized_textblob_Polarity   Model B: normalized_SentiWordNet_scores   0.19771060946960928

Model A: normalized_vader_Polarity   Model B: normalized_AFINN_scores   0.5553993980731224

Model A: normalized_vader_Polarity   Model B: normalized_SentiWordNet_scores   0.13575372460319957

Model A: normalized_AFINN_scores   Model B: normalized_SentiWordNet_scores   0.21681130425251294



### How do vader and afinn agree with annotations?

In [83]:
# agreement on positives peek:
afinn_agree_shape = df.loc[
    (df['normalized_AFINN_scores'] > 0.5) &
    (df['our_label'] == 'positive') 
].shape

vader_agree_shape = df.loc[
    (df['normalized_vader_Polarity'] > 0.5) &
    (df['our_label'] == 'positive') 
].shape


# agreement on negatives peek:
afinn_agree_neg_shape = df.loc[
    (df['normalized_AFINN_scores'] < 0.5) &
    (df['our_label'] == 'negative') 
].shape

vader_agree_neg_shape = df.loc[
    (df['normalized_vader_Polarity'] < 0.5) &
    (df['our_label'] == 'negative') 
].shape


# Our shapes
our_pos_shape = df.loc[df['our_label'] == 'positive'].shape
our_pos_neut_shape = df.loc[
    (df['our_label'] == 'positive') |
    (df['our_label'] == 'neutral')
].shape
our_neg_shape = df.loc[df['our_label'] == 'negative'].shape


afinn_agree_shape[0] / our_pos_shape[0]  # 0.4411764705882353
vader_agree_shape[0] / our_pos_shape[0]  # 0.5294117647058824
afinn_agree_neg_shape[0] / our_neg_shape[0]  # 0.7741935483870968
vader_agree_shape[0] / our_neg_shape[0]  # 0.1935483870967742

0.1935483870967742

In [86]:
# Back up my data
df.to_csv('../../data/normalized_annotatted_all_2023-04-08_20.20.00.0000.tsv', sep='\t')

In [90]:
%%sh
# Random forest classifier?

ls -lrt ../../data/ | grep .tsv


-rw-r--r--  1 melissacirtain  staff   1699708 Mar 26 14:46 #chrisrocklive_project_tweets.2023-03-09_17.16.18.676751.tsv
-rw-r--r--  1 melissacirtain  staff   1697993 Mar 26 14:46 #tyrenichols_project_tweets.2023-03-09_19.25.43.435891.tsv
-rw-r--r--  1 melissacirtain  staff   1646638 Mar 26 14:46 chatgpt_project_tweets.2023-03-09_19.12.39.616553.tsv
-rw-r--r--  1 melissacirtain  staff  14558535 Mar 26 14:46 deduplicated_tweets.2023-03-26_14.08.48.636120.tsv
-rw-r--r--  1 melissacirtain  staff    381596 Mar 26 14:46 maga_project_tweets.2023-03-08_22.25.11.437694.tsv
-rw-r--r--  1 melissacirtain  staff   1724693 Mar 26 14:46 obama_project_tweets.2023-03-08_22.36.15.226664.tsv
-rw-r--r--  1 melissacirtain  staff   1773671 Mar 26 14:46 russia_project_tweets.2023-03-08_22.51.51.710826.tsv
-rw-r--r--  1 melissacirtain  staff   1762648 Mar 26 14:46 statehood_project_tweets.2023-03-09_17.12.00.547177.tsv
-rw-r--r--  1 melissacirtain  staff   6029631 Mar 26 14:46 textblob_polarity.2023-03-26_14.

In [95]:
test_df = pd.read_csv('../../data/normalized_annotated_all_2023-04-07_12.43.49.255973.tsv', delimiter='\t')
test_df.columns
test_df.loc[~test_df['our_label'].isna()].head(3)

,Unnamed: 0,Unnamed: 0.1,conversation_id,lang,reply_settings,created_at,clean_text,text,author_id,referenced_tweets,...,AFINN_sentiments,SentiWordNet_scores,normalized_textblob_Polarity,normalized_textblob_Subjectivity,normalized_vader_Polarity,normalized_vader_Subjectivity,normalized_AFINN_scores,normalized_SentiWordNet_scores,our_label,annotator
185,185,186,1633724515212742656,en,everyone,2023-03-09 07:00:52+00:00,rt netflix chrisrocklive tonight pm et pm pt m...,RT @netflix: #ChrisRockLive TONIGHT at 10pm ET...,2348520354,"[{'type': 'retweeted', 'id': '1631421624359739...",...,negative,0.0,0.623918,0.468254,0.781464,0.364955,0.469388,0.412698,neutral,anu
285,285,286,1633637250419367936,en,everyone,2023-03-09 01:14:06+00:00,must read important notice different fixed mat...,YOU MUST READ ‼️‼️‼️‼️\nIMPORTANT NOTICE DIFF...,1631666639862132736,NaN,...,positive,1.0,0.583333,0.600000,0.601621,0.107143,0.530612,0.539683,neutral,melissa
359,359,360,1633586187700150272,en,everyone,2023-03-08 21:51:12+00:00,rt netflixisajoke shout ohsnapjbsmoove flying ...,RT @NetflixIsAJoke: Shout out to @ohsnapjbsmoo...,852279096310546432,"[{'type': 'retweeted', 'id': '1633573306157916...",...,neutral,0.0,0.500000,0.000000,0.702710,0.191964,0.489796,0.412698,positive,anu
